In [14]:
import requests
import json
import pandas as pd

# =============================================================================
# CONFIGURAÇÃO
# =============================================================================
EMAIL = "diego_moura@cargill.com"
SENHA = "Cargill@25"

BASE_URL = "https://apidtz.comexia.digital/api"

# =============================================================================
# 1) AUTENTICAÇÃO
# =============================================================================
def autenticar():
    url = f"{BASE_URL}/auth"
    payload = {"email": EMAIL, "senha": SENHA}
    resp = requests.post(url, json=payload)
    resp.raise_for_status()
    data = resp.json()
    token = data.get("access_token") or data.get("token")
    if not token:
        raise ValueError("Token de autenticação ausente na resposta")
    print("✅ Autenticado com sucesso!")
    return token

# =============================================================================
# 2) SOLICITAR MONITORAMENTO DE NAVIO
# =============================================================================
def solicitar_monitoramento(token):
    url = f"{BASE_URL}/monitor/navio"
    payload = {
        "cnpj": "60.498.706/0001-57",
        "lista": [
            {
                "cnpj_terminal": "02.762.121/0001-04",
                "nome_navio": "COSCO SHIPPING URUGUAY",
                "viagem_navio": "006E"
            }
        ]
    }
    headers = {"Authorization": f"Bearer {token}", "Content-Type": "application/json"}
    resp = requests.post(url, headers=headers, json=payload)
    if resp.status_code >= 400:
        body_text = (resp.text or "").lower()
        if "already exist" in body_text or "already tracked" in body_text:
            print("ℹ️ Navio já está sendo monitorado. Prosseguindo para visualização...")
            return "already_exists"
        print(f"⚠️ Erro ao solicitar monitoramento ({resp.status_code}): {resp.text[:300]}")
        return None
    print("📡 Monitoramento solicitado:")
    try:
        print(json.dumps(resp.json(), indent=2, ensure_ascii=False))
    except Exception:
        print(resp.text)
    return "ok"



In [15]:
# =============================================================================
# 3) VISUALIZAR MONITORAMENTOS
# =============================================================================
def visualizar_monitoramentos(token):
    url = f"{BASE_URL}/terminalmonitorings"
    payload = {
        "cnpj": "60.498.706/0001-57",
        "cnpj_terminal": "02.762.121/0001-04",
        "nome_navio": "COSCO SHIPPING URUGUAY",
        "viagem_navio": "006E"
    }
    headers = {"Authorization": f"Bearer {token}", "Content-Type": "application/json"}
    resp = requests.post(url, headers=headers, json=payload)
    if resp.status_code >= 400:
        print(f"⚠️ Erro ao visualizar monitoramentos ({resp.status_code}): {resp.text[:300]}")
        return pd.DataFrame()
    data = resp.json() if resp.content else []
    print("📊 Monitoramentos encontrados:", len(data) if isinstance(data, list) else 0)

    # Criar DataFrame
    df = pd.DataFrame(data) if isinstance(data, list) else pd.DataFrame()
    if not df.empty:
        print(df.head())  # Mostra as primeiras linhas
    return df

# =============================================================================
# MAIN
# =============================================================================
if __name__ == "__main__":
    token = autenticar()
    status = solicitar_monitoramento(token)

    # Se já está monitorado ou foi solicitado com sucesso, prossegue para visualizar
    if status in ("ok", "already_exists"):
        df_monitoramentos = visualizar_monitoramentos(token)
        
        # Salvar no Oracle (F_ELLOX_TERMINAL_MONITORINGS)
        if not df_monitoramentos.empty:
            try:
                from database import upsert_terminal_monitorings_from_dataframe
                processed = upsert_terminal_monitorings_from_dataframe(df_monitoramentos)
                print(f"✅ {processed} registros salvos/atualizados no Oracle (F_ELLOX_TERMINAL_MONITORINGS)")
            except Exception as e:
                print(f"❌ Erro ao salvar no Oracle: {e}")
        
        # Se quiser salvar em Excel
        if not df_monitoramentos.empty:
            df_monitoramentos.to_excel("monitoramentos.xlsx", index=False)
            print("💾 Dados salvos em monitoramentos.xlsx")
    else:
        print("❌ Não foi possível prosseguir para a visualização")


✅ Autenticado com sucesso!
ℹ️ Navio já está sendo monitorado. Prosseguindo para visualização...
📊 Monitoramentos encontrados: 10
         id                   navio viagem agencia         data_deadline  \
0  39097145  COSCO SHIPPING URUGUAY   006E          2025-08-21T11:23:00Z   
1  39094997  COSCO SHIPPING URUGUAY   006E          2025-08-21T11:23:00Z   
2  39092848  COSCO SHIPPING URUGUAY   006E          2025-08-21T11:23:00Z   
3  39090734  COSCO SHIPPING URUGUAY   006E          2025-08-21T11:23:00Z   
4  39089257  COSCO SHIPPING URUGUAY   006E          2025-08-21T11:23:00Z   

  data_draft_deadline data_abertura_gate data_abertura_gate_reefer  \
0                                                                    
1                                                                    
2                                                                    
3                                                                    
4                                                               

In [ ]:
# =============================================================================
# 4) TESTE DIRETO DO ENDPOINT /monitorings (pedido do usuário)
# =============================================================================
import requests, json

def testar_monitorings():
    token = autenticar()
    headers = {"Authorization": f"Bearer {token}", "Content-Type": "application/json"}
    url = f"{BASE_URL}/monitorings"
    payload = {
        "cnpj": "70.098.822/0001-32",
        "cnpj_terminal": "04.887.625/0001-78",
        "nome_navio": "MSC BOSPHORUS",
        "viagem_navio": "UA252R"
    }
    resp = requests.post(url, headers=headers, json=payload, timeout=30)
    print(resp.status_code)
    try:
        data = resp.json()
    except Exception:
        print(resp.text)
        return None
    # Imprimir chaves do primeiro registro, se houver
    if isinstance(data, list) and data:
        print("Chaves no 1º registro:", list(data[0].keys()))
    else:
        print("Retorno não é lista ou está vazio.")
    print(json.dumps(data[:2] if isinstance(data, list) else data, indent=2, ensure_ascii=False))
    return data

if __name__ == "__main__":
    testar_monitorings()
